In [1]:
# -*- coding: UTF-8 -*-
import json
import os
from os.path import join as pathj
import datetime
# from loguru import logger
from PIL import Image
import cv2

def ensure_dir(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    return folder

def line2str(l):
    str1=''
    for ele in l[0:-1]:
        str1 = str1 + str(ele)+' '
    return str1 + str(l[-1])

def gen_class2id(classes):
    cls2id = {}
    for k,v in enumerate(classes):
        cls2id[v] = k
    return cls2id

def xyxy2xywh(box):
    x1 = min(box[0], box[2])
    y1 = min(box[1], box[3])
    w = abs(box[2]-box[0])
    h = abs(box[3]-box[1])
    return [x1, y1, w, h, w*h]

def gen_empty_cocojson_instance(class_name_en):
    # 创建 总标签data 
    now = datetime.datetime.now()
    instance = dict(
        info=dict(
            description=None,
            url=None,
            version=None,
            year=now.year,
            contributor=None,
            date_created=now.strftime("%Y-%m-%d %H:%M:%S.%f"),
        ),
        licenses=[dict(url=None, id=0, name=None,)],
        images=[
            # license, url, file_name, height, width, date_captured, id
        ],
        type="instances",
        annotations=[
            # segmentation, area, iscrowd, image_id, bbox, category_id, id
        ],
        categories = gen_category_dict(class_name_en),
        # categories=[
        #     # supercategory, id, name
        # ],
    )

    return instance

def gen_one_image_dict(img_name, h, w, id):
    data = {
        'file_name': os.path.basename(img_name),
        'height': h,
        'width': w,
        'id': id        
    }
    return data

def gen_one_anno_dict(cat_id, img_id, bbox: list, area, anno_id):
    anno = {
        'segmentation':[],
        'area': area, #直接取绝对值
        'iscrowd': 0,
        'image_id': img_id,
        'bbox': bbox,
        'category_id': cat_id,   
        'id': anno_id

    }
    return anno
    
def gen_category_dict(class_name):
    ins = []
    for k,v in enumerate(class_name):
        one_cate = {'id':k, 'name':v,'supercategory':v}
        ins.append(one_cate) 
    return ins

def read_txt(abs_txt_name):
    with open(abs_txt_name,'r', encoding='utf-8') as f:
        for line in f.readlines():
                line = line.replace(')','').replace('(','')
                row = line.strip().split(',')
                print(row)
# 读取json
def read_json(abs_txt_name):
    with open(abs_txt_name, 'r', encoding='utf-8') as f: #读文件名
        ret_dic = json.load(f)
        return ret_dic
        
def save_json(new_name,json_ins):
    with open(new_name, 'w',encoding='utf-8') as f:
        json.dump(json_ins ,f ,indent=2,ensure_ascii=False)#zw
        print('{} saved'.format(new_name)) 

In [5]:
tmp_dir =r'D:\sa_data\0011_v1.6_filtered_linebox_yolox\labelme\imgs'
img_names = [x for x in os.listdir(pathj(tmp_dir, './')) if x.endswith('png')]
json_names = [x for x in os.listdir(pathj(tmp_dir, './')) if x.endswith('json')]

text_list = ['icontext','text','icon_text','text_icon_','icon_question_'] # text_icon的所有系类别
image_list = ['image', 'icon','icon_']
# op = ['add', 'arrow','close','cross','magnifier', 'minimize','maximize','question','search','selectBox']
linebox_list =['inputbox', 'tab', 'linebox'] #, 'scrollbar_slider','track'
line_list = ['v_segline', 'h_segline', 'scrollbar_slider', 'track']
#labelme目录中 同时有图片和json
cate2finecate={'text':text_list, 'icon':image_list, 'linebox': linebox_list}
# cate2finecate4={'text':text_list, 'image':image_list, 'linebox': linebox_list, 'line': line_list}
cate2finecate4={'linebox': linebox_list, 'line': line_list}
class_name_en_3 = ['text', 'icon', 'linebox'] # 暂时还是叫icon
# class_name_en_4 = ['text', 'image', 'linebox', 'line']
# class_name_en_4 = ['linebox', 'line']
# text如果满足了就是 text的其余都是image的 icon
def finecate_to_basecate(cls, cls_map:dict):
    keys = cls_map.keys()
    # print(keys)
    for key in keys:
        for i in cls_map[key]:
            if cls == i or i in cls:
                return key
    return 'no'


In [6]:

cls = set()
label_list =[]
for name in json_names:
    ins = read_json(pathj(tmp_dir, name))
    for one in ins['shapes']:
        cate = finecate_to_basecate(one['label'], cate2finecate)
        if cate == 'no':
            continue
        else:
            cls.add(cate)
            label_list.append(cate)

In [7]:
from collections import Counter
count =Counter(label_list)
print(count)

Counter({'text': 793792, 'icon': 455580, 'linebox': 40364})


In [8]:
import numpy as np

file_names = []
for name in img_names:
    file_names.append(name)
file_names = np.array(file_names)
np.random.seed(777)
np.random.shuffle(file_names)
# 0.85 : 0.15
end = int(len(file_names)*0.85)

t_name = file_names[:end].tolist()
v_name = file_names[end:].tolist()
#打乱

In [17]:

coco_ins = gen_empty_cocojson_instance(class_name_en_3)

import copy
val_coco_ins = copy.deepcopy(coco_ins)
# print(coco_ins['categories'])
# os.path.basename('sdsd\sdsds\asdasd\l.ll')
class_name_en_3.index('linebox')


2

In [18]:
#--- folder path：data目录
root_dir =r'D:\sa_data\0011_v1.6_filtered_linebox_yolox\labelme\imgs'
# D:\sa_data\other\NWPU VHR-10 dataset\gtwithimg
#'#保存路径 images,labels

img_part = coco_ins['images']
anno_part = coco_ins['annotations']

name_id = 0
anno_id = 0
anno_count =0
max_det= 0
min_det =10

json_names = [x for x in os.listdir(root_dir) if x.endswith('json')]
#遍历每个 json 文件
for name in json_names:

    file_path = os.path.join(root_dir,name)
    img_name = file_path[:-5]+'.png'
    if name[:-5]+'.png' not in t_name: 
        continue
    txt =[]
    img = cv2.imread(img_name, flags=cv2.IMREAD_COLOR)
    # images
    name_id += 1
    data_ins = gen_one_image_dict(img_name, img.shape[0], img.shape[1], name_id)
    img_part.append(data_ins) #宽高、名字
    # img = Image.open(img_name)#txt怎么取名的这个问题
    # load labelme json
    labelme_ins = read_json(file_path)
    shapes = labelme_ins['shapes']
    per_img_annos_count = 0
    for shape in shapes:
        # print(shape)
        cate = finecate_to_basecate(shape['label'], cate2finecate)
        if cate == 'no':
            continue
        else:
            x1,y1 = shape['points'][0]
            x2,y2 = shape['points'][1]
            bbox =xyxy2xywh([x1,y1,x2,y2])
            # label_list.append(cate)
            cls_id = class_name_en_3.index(cate) # 最后一行纯数字 id从1开始
            anno_id+=1
            anno_ins = gen_one_anno_dict(cls_id, name_id, bbox[:4], bbox[4], anno_id)
            anno_part.append(anno_ins) #宽高、名字
            per_img_annos_count+=1

    # if per_img_annos_count>0:
    #     name_id += 1
    #     data_ins = gen_one_image_dict(img_name, img.shape[0], img.shape[1], name_id)
    #     anno_part.append(data_ins)

print('total transform: %s'% (anno_id))
print(f'min:{min_det}, max:{max_det}')

total transform: 1091065
min:10, max:0


In [13]:
# 验证集
val_img_part = val_coco_ins['images']
val_anno_part = val_coco_ins['annotations']

name_id = 0
anno_id = 0
anno_count =0
max_det= 0
min_det =10

json_names = [x for x in os.listdir(root_dir) if x.endswith('json')]
#遍历每个 json 文件
for name in json_names:

    file_path = os.path.join(root_dir,name)
    img_name = file_path[:-5]+'.png'
    if name[:-5]+'.png' not in v_name: # if name in trainning set
        continue
    txt =[]
    img = cv2.imread(img_name, flags=cv2.IMREAD_COLOR)
    # img = Image.open(img_name)#txt怎么取名的这个问题
    name_id += 1
    data_ins = gen_one_image_dict(img_name, img.shape[0], img.shape[1], name_id)
    val_img_part.append(data_ins) #宽高、名字
    # load labelme json
    labelme_ins = read_json(file_path)
    shapes = labelme_ins['shapes']
    for shape in shapes:
        # print(shape)
        cate = finecate_to_basecate(shape['label'], cate2finecate)
        if cate == 'no':
            continue
        else:
            x1,y1 = shape['points'][0]
            x2,y2 = shape['points'][1]
            bbox =xyxy2xywh([x1,y1,x2,y2])
            # label_list.append(cate)
            cls_id = class_name_en_3.index(cate) # 最后一行纯数字 id从1开始
            anno_id+=1
            anno_ins = gen_one_anno_dict(cls_id, name_id, bbox[:4], bbox[4], anno_id)
            val_anno_part.append(anno_ins) #宽高、名字
            
        img_part.append(data_ins) #宽高、名字
print('total transform: %s'% (anno_id))
print(f'min:{min_det}, max:{max_det}')

total transform: 198671
min:10, max:0


In [82]:
print(val_coco_ins['images'])

[]


In [14]:

save_dir = root_dir
save_json(pathj(save_dir, '../',"val_v1.6.json"), val_coco_ins)

D:\sa_data\0011_v1.6_filtered_linebox_yolox\labelme\imgs\../val_v1.6.json saved


In [19]:
save_dir = root_dir
save_json(pathj(save_dir, '../',"train_v1.6.json"), coco_ins)

D:\sa_data\0011_v1.6_filtered_linebox_yolox\labelme\imgs\../train_v1.6.json saved


In [87]:
import yaml

c = {'names':{}}
for k,v in enumerate(class_name_en):
    c['names'][k] = v
print(yaml.dump(c, allow_unicode=True))
with open('b1.yaml', 'w') as file:
    file.write(yaml.dump(c, allow_unicode=True))

names:
  0: text
  1: image
  2: linebox

